In [ ]:
# import pandas as pd
# from google.colab import drive

# drive.mount('/content/drive')
# base = "/content/drive/MyDrive/FIT5196Assignment2/" # for colab

In [1]:
# begin here if running locally

# for local drive
base = ""

In [3]:

import pandas as pd
warehouses = pd.read_csv(base + 'warehouses.csv')

In [4]:


dirty_data = pd.read_csv(base + 'Group_035_dirty_data.csv')
print(dirty_data)

      order_id   customer_id        date nearest_warehouse  \
0    ORD257850  ID0767584756  2019-07-13          Thompson   
1    ORD216249  ID2247272473  2019-07-11         Nickolson   
2    ORD316891  ID6167441075  2019-06-22         Nickolson   
3    ORD040572  ID2189145378  2019-07-16         Nickolson   
4    ORD091929  ID0145235237  2019-06-06         Nickolson   
..         ...           ...         ...               ...   
495  ORD329999  ID0615176175  2019-11-09            Bakers   
496  ORD379470  ID0287629420  2019-04-27          Thompson   
497  ORD000718  ID3313134375  2019-12-11         Nickolson   
498  ORD201885  ID0026051952  2019-02-07         Nickolson   
499  ORD089659  ID0575375067  2019-07-18         Nickolson   

                                         shopping_cart  order_price  \
0               [('iAssist Line', 1), ('Alcon 10', 1)]        11175   
1    [('Universe Note', 2), ('pearTV', 2), ('Toshik...        29020   
2    [('Universe Note', 2), ('Thunder line

In [11]:
# Outlier data
outlier_data = pd.read_csv(base + 'Group_035_outlier_data.csv')

In [15]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

# --- Assume you have already loaded ---
df = outlier_data.copy()

# --- Step 1: Prepare features ---
X = df[['distance_to_nearest_warehouse', 'is_expedited_delivery', 'is_happy_customer', 'season']]
y = df['delivery_charges']

# Convert boolean to integer
X['is_expedited_delivery'] = X['is_expedited_delivery'].astype(int)
X['is_happy_customer'] = X['is_happy_customer'].astype(int)

# One-hot encode season (drop first to avoid multicollinearity)
encoder = OneHotEncoder(drop='first', sparse_output=False)
season_encoded = encoder.fit_transform(X[['season']])
season_encoded_df = pd.DataFrame(season_encoded, columns=encoder.get_feature_names_out(['season']))

# Combine base features and season dummies
X_base = X.drop(columns=['season']).reset_index(drop=True)
X_encoded = pd.concat([X_base, season_encoded_df], axis=1)

# --- Step 2: Add interaction terms between season and numeric predictors ---
for season_col in season_encoded_df.columns:
    for feature in ['distance_to_nearest_warehouse', 'is_expedited_delivery', 'is_happy_customer']:
        interaction_name = f"{feature}_x_{season_col}"
        X_encoded[interaction_name] = X_base[feature] * season_encoded_df[season_col]

# --- Step 3: Fit linear model ---
model = LinearRegression()
model.fit(X_encoded, y)

# --- Step 4: Compute predictions and residuals ---
df['predicted_delivery_charge'] = model.predict(X_encoded)
df['residual'] = df['delivery_charges'] - df['predicted_delivery_charge']

# --- Step 5: Standard deviation–based outlier detection ---
std_resid = df['residual'].std()
outlier_std_mask = np.abs(df['residual']) > 3 * std_resid
outliers_std = df[outlier_std_mask]

print("=== Outliers using standard deviation rule ===")
print(outliers_std[['order_id', 'delivery_charges', 'predicted_delivery_charge', 'residual']])

# --- Step 6: Robust Z-score (Median + MAD) method ---
median_resid = df['residual'].median()
mad_resid = np.median(np.abs(df['residual'] - median_resid))
robust_z = 0.6745 * (df['residual'] - median_resid) / mad_resid

outlier_robust_mask = np.abs(robust_z) > 3.5   # Common robust z-score threshold
outliers_robust = df[outlier_robust_mask]

print("\n=== Outliers using robust z-score ===")
print(outliers_robust[['order_id', 'delivery_charges', 'predicted_delivery_charge', 'residual',]])

# --- Step 7: Optionally filter outliers ---
filtered_std = df[~outlier_std_mask].reset_index(drop=True)
filtered_robust = df[~outlier_robust_mask].reset_index(drop=True)

print(f"\nRemoved {outliers_std.shape[0]} outliers (std-based), remaining: {filtered_std.shape[0]}")
print(f"Removed {outliers_robust.shape[0]} outliers (robust-based), remaining: {filtered_robust.shape[0]}")


=== Outliers using standard deviation rule ===
      order_id  delivery_charges  predicted_delivery_charge   residual
17   ORD089831           144.450                  96.864118  47.585882
93   ORD276933            39.570                  76.640083 -37.070083
98   ORD142753            39.010                  73.782575 -34.772575
109  ORD233575           119.955                  80.004734  39.950266
123  ORD099918            39.790                  80.272405 -40.482405
137  ORD272158            33.335                  64.123499 -30.788499
143  ORD414922           143.385                  97.169650  46.215350
145  ORD482557           164.775                 112.698279  52.076721
167  ORD141312           126.705                  88.620992  38.084008
193  ORD204921            40.450                  80.686767 -40.236767
194  ORD318956           193.875                 145.804701  48.070299
207  ORD285918            52.185                 106.343889 -54.158889
252  ORD181666           167.4

C:\Users\adria\AppData\Local\Temp\ipykernel_6676\1690034405.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['is_expedited_delivery'] = X['is_expedited_delivery'].astype(int)
C:\Users\adria\AppData\Local\Temp\ipykernel_6676\1690034405.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['is_happy_customer'] = X['is_happy_customer'].astype(int)


In [6]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

df = outlier_data.copy()

# --- Step 1: Prepare data for modeling ---
X = df[['distance_to_nearest_warehouse', 'is_expedited_delivery', 'is_happy_customer', 'season']]
y = df['delivery_charges']

# Convert boolean to int (0/1)
X['is_expedited_delivery'] = X['is_expedited_delivery'].astype(int)
X['is_happy_customer'] = X['is_happy_customer'].astype(int)

# One-hot encode the categorical 'season'
encoder = OneHotEncoder(drop='first', sparse_output=False)
season_encoded = encoder.fit_transform(X[['season']])
season_encoded_df = pd.DataFrame(season_encoded, columns=encoder.get_feature_names_out(['season']))
X_encoded = pd.concat([X.drop(columns=['season']).reset_index(drop=True), season_encoded_df], axis=1)

# --- Step 2: Fit linear regression model ---
model = LinearRegression()
model.fit(X_encoded, y)

# --- Step 3: Compute predicted values and residuals ---
df['predicted_delivery_charge'] = model.predict(X_encoded)
df['residual'] = df['delivery_charges'] - df['predicted_delivery_charge']

# --- Step 4: Identify outliers based on residuals ---
# Define outliers as points more than 3 standard deviations away
std_resid = df['residual'].std()
outlier_mask = np.abs(df['residual']) > 3 * std_resid
outliers = df[outlier_mask]

# --- Step 5: Print outliers ---
print("Detected outliers based on delivery_charge residuals:")
print(outliers[['order_id', 'delivery_charges', 'predicted_delivery_charge', 'residual']])

# --- Step 6: Remove outliers ---
filtered_df = df[~outlier_mask].reset_index(drop=True)

print(f"\nRemoved {outliers.shape[0]} outliers. Remaining records: {filtered_df.shape[0]}")


Detected outliers based on delivery_charge residuals:
      order_id  delivery_charges  predicted_delivery_charge   residual
17   ORD089831           144.450                  94.770491  49.679509
93   ORD276933            39.570                  79.393501 -39.823501
98   ORD142753            39.010                  75.439337 -36.429337
109  ORD233575           119.955                  84.049439  35.905561
123  ORD099918            39.790                  83.044732 -43.254732
143  ORD414922           143.385                  95.117816  48.267184
145  ORD482557           164.775                 105.864411  58.910589
167  ORD141312           126.705                  88.983992  37.721008
193  ORD204921            40.450                  83.339513 -42.889513
194  ORD318956           193.875                 129.416621  64.458379
207  ORD285918            52.185                 101.343840 -49.158840
252  ORD181666           167.445                 107.500402  59.944598
273  ORD429584         

C:\Users\adria\AppData\Local\Temp\ipykernel_6676\3947782600.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['is_expedited_delivery'] = X['is_expedited_delivery'].astype(int)
C:\Users\adria\AppData\Local\Temp\ipykernel_6676\3947782600.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['is_happy_customer'] = X['is_happy_customer'].astype(int)


I have two pandas datasets, warehouses and outlier_data, with the following columns: 
1. warehouses has the following columns: 
names,lat,lon 

2. outlier_data has the following columns: 
order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer 

i want to detect and remove outliers in delivery_charge. delivery_charge depends linearly on: distance_to_nearest_warehouse,is_expedited_delivery,is_happy_customer, and these 3 variables vary differently depending on season. 

to solve this, use sklearn.linear_model.LinearRegression to build a linear model to determine delivery_charge. distance_to_nearest_warehouse is a positive float, is_expedited_delivery and is_happy_customer are booleans. season is a categorical variable with 4 values, that modifies the weights of the other 3 variables. after building the linear model to determine delivery_charge, identify which values are considered outliers as compared to the predicted value. print out these outliers first, then filter them out.

In [2]:

missing_data = pd.read_csv(base + 'Group_035_missing_data.csv')
print(missing_data.isna().sum())



NameError: name 'pd' is not defined

In [4]:
import ast
import numpy as np

# Parse shopping_cart into list of tuples
def parse_cart(cart_str):
    if pd.isna(cart_str):
        return []
    try:
        return ast.literal_eval(cart_str)
    except Exception:
        return []

missing_data['parsed_cart'] = missing_data['shopping_cart'].apply(parse_cart)

# Collect unique items
all_items = sorted({item for cart in missing_data['parsed_cart'] for item, _ in cart})
item_index = {item: i for i, item in enumerate(all_items)}

# Build system of equations A x = b
rows, b = [], []
for _, row in missing_data.iterrows():
    if pd.notna(row['order_price']) and row['parsed_cart']:
        vec = np.zeros(len(all_items))
        for item, qty in row['parsed_cart']:
            vec[item_index[item]] += qty
        rows.append(vec)
        b.append(row['order_price'])

A = np.vstack(rows)
b = np.array(b)

# Solve least squares for unit prices
x, _, _, _ = np.linalg.lstsq(A, b, rcond=None)

# Round to 2 decimals for catalog
x = np.round(x, 2)

# Build catalog
catalog = {item: price for item, price in zip(all_items, x)}


In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# --- Haversine helper ---
def haversine(lat1, lon1, lat2, lon2):
    R = 6378  # Earth radius in KM
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

# --- Prepare Sentiment analyzer ---
sia = SentimentIntensityAnalyzer()

# --- Imputation logic ---
def impute_row(row, warehouses, catalog):
    # --- Impute nearest_warehouse ---
    if pd.isna(row['nearest_warehouse']) and pd.notna(row['customer_lat']) and pd.notna(row['customer_long']):
        distances = warehouses.apply(
            lambda wh: haversine(row['customer_lat'], row['customer_long'], wh['lat'], wh['lon']), axis=1
        )
        nearest_idx = distances.idxmin()
        row['nearest_warehouse'] = warehouses.loc[nearest_idx, 'names']
        row['distance_to_nearest_warehouse'] = distances.min()

    # --- Impute order_price ---
    if pd.isna(row['order_price']):
        if pd.notna(row['order_total']) and pd.notna(row['delivery_charges']) and pd.notna(row['coupon_discount']):
            denom = (100 - row['coupon_discount']) / 100
            if denom != 0:
                row['order_price'] = (row['order_total'] - row['delivery_charges']) / denom
        elif pd.isna(row['order_total']) and row['parsed_cart']:  # fallback to catalog
            row['order_price'] = sum(catalog.get(item, 0) * qty for item, qty in row['parsed_cart'])

    # --- Impute delivery_charges ---
    if pd.isna(row['delivery_charges']):
        if pd.notna(row['order_total']) and pd.notna(row['order_price']) and pd.notna(row['coupon_discount']):
            denom = (100 - row['coupon_discount']) / 100
            row['delivery_charges'] = row['order_total'] - row['order_price'] * denom

    # --- Impute order_total ---
    if pd.isna(row['order_total']):
        if pd.notna(row['order_price']) and pd.notna(row['coupon_discount']) and pd.notna(row['delivery_charges']):
            denom = (100 - row['coupon_discount']) / 100
            row['order_total'] = row['order_price'] * denom + row['delivery_charges']

    # --- Impute distance_to_nearest_warehouse ---
    if pd.isna(row['distance_to_nearest_warehouse']):
        if pd.notna(row['nearest_warehouse']) and pd.notna(row['customer_lat']) and pd.notna(row['customer_long']):
            wh = warehouses.loc[warehouses['names'] == row['nearest_warehouse']].iloc[0]
            row['distance_to_nearest_warehouse'] = haversine(row['customer_lat'], row['customer_long'], wh['lat'], wh['lon'])

    # --- Impute is_happy_customer from sentiment---
    if pd.isna(row['is_happy_customer']) and pd.notna(row['latest_customer_review']):
        sentiment = sia.polarity_scores(str(row['latest_customer_review']))
        row['is_happy_customer'] = sentiment['compound'] >= 0.05

    return row

# Apply
missing_data_imputed = missing_data.apply(lambda r: impute_row(r, warehouses, catalog), axis=1)


In [ ]:

print(missing_data_imputed.isna().sum())
rows_with_nulls = missing_data_imputed[missing_data_imputed.isnull().any(axis=1)]
print(rows_with_nulls)

print(catalog)

order_id                         0
customer_id                      0
date                             0
nearest_warehouse                0
shopping_cart                    0
order_price                      0
delivery_charges                 0
customer_lat                     0
customer_long                    0
coupon_discount                  0
order_total                      0
season                           0
is_expedited_delivery            0
distance_to_nearest_warehouse    0
latest_customer_review           0
is_happy_customer                0
parsed_cart                      0
dtype: int64
{'Alcon 10': np.float64(8950.0), 'Candle Inferno': np.float64(430.0), 'Lucent 330S': np.float64(1230.0), 'Olivia x460': np.float64(1225.0), 'Thunder line': np.float64(2180.0), 'Toshika 750': np.float64(4320.0), 'Universe Note': np.float64(3450.0), 'iAssist Line': np.float64(2225.0), 'iStream': np.float64(150.0), 'pearTV': np.float64(6310.0)}


I have two pandas datasets, warehouses and missing_data, with the following columns:
1. warehouses has the following columns:
names,lat,lon

2. missing_data has the following columns:
order_id,customer_id,date,nearest_warehouse,shopping_cart,order_price,delivery_charges,customer_lat,customer_long,coupon_discount,order_total,season,is_expedited_delivery,distance_to_nearest_warehouse,latest_customer_review,is_happy_customer

I want to perform data imputation. The columns which have missing entries, and the imputing strategy for each column, are:
nearest_warehouse: compute the lowest Haversine Distance (with radius of earth = 6378 KM) between the customer_lat, customer_long and each of the lat, lon pairs in warehouses, and impute with the "names" of the lowest Haversine Distance.
order_price: (order_total - delivery_charges)/((100-coupon_discount)/100)
delivery_charges: order_total - order_price*((100-coupon_discount)/100)
order_total: order_price*((100-coupon_discount)/100) + delivery_charges
distance_to_nearest_warehouse: Haversine Distance (with radius of earth = 6378 KM) of customer_lat, customer_long vs the lat, lon of nearest_warehouse as found in warehouses
is_happy_customer: check latest_customer_review with SentimentIntensityAnalyzer from 
nltk.sentiment.vader to obtain the polarity score. A sentiment is considered positive if it has a 'compound' polarity score of 0.05 or higher and is considered negative otherwise.

If any row has too many missing columns to do any of the above imputation, do not impute it so that i can later see what's incomplete.


shopping_cart, a column in missing_data is a list of tuples, of (Item_ordered, Quantity), example: "[('Item 330S', 1), ('sampleItem', 2)]". 

there are 10 categories of items. using the column shopping_cart, first identify the 10 categories of items, then use numpy.linalg to determine the unit price of each item.

store the values of this in a dictionary named catalog.

in the case that both order_price and order_total are missing, order_price should be imputed from shopping_cart. use the number of each type of item ordered in shopping_cart, and, referencing the values found in catalog calculated earlier, impute order_price from this. order_total then can be imputed as normal.




In [ ]:

outlier_data = pd.read_csv(base + 'Group_035_outlier_data.csv')
print(outlier_data)
